In [1]:
# Required python libraries         
import numpy as np         
import os                  
from random import shuffle
import random
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm 
import time
import pickle
from tqdm import tqdm
from statistics import mode


# OpenCV and scikit-learn
from sklearn.utils import resample
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances, cosine_distances
from sklearn.metrics import silhouette_score
# from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans
import cv2 

# Pandas 
# import pandas as pd

# Tensorflow
# import tensorflow as tf
import pandas as pd
# from tensorflow.keras.applications.densenet import DenseNet169
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.resnet import ResNet101 
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# Laoading the pickle file (list of dictionary) consisting of features of image obtained from the mentioned model
with open('../../pickle_files/al/ct_scan/ct_scan_resnet101.pickle','rb') as handle:
   t_dataset  = pickle.load(handle)  # Type of t_dataset: python list

In [5]:
# Shuffling the loaded list
random.seed(42)
shuffle(t_dataset)

In [6]:
# View the first element
t_dataset[0]

{'id': 3771,
 'filepath': './ct_scan_dataset/3A_images/NCP_509_2175_0024.png',
 'image': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 'label': 1}

In [7]:
# def sub_clusters(features):
#     sum_of_squared_distances = []
#     no_cluster=[]
#     K=range(2,70,2)
#     for num_clusters in K:
#         kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init="auto")
#         kmeans.fit(features)
#         no_cluster.append(num_clusters)
#         sum_of_squared_distances.append(kmeans.inertia_)
#     print(sum_of_squared_distances)
#     output= kmeans.labels_
#     clusters = [np.squeeze(np.array(features)[[np.where(output==i)[0]]],axis=0) for i in range(len(np.unique(output)))]
#     return kmeans.cluster_centers_, clusters

In [8]:
# Method to create subclusters 
def sub_clusters(features):
    kmeans = KMeans(n_clusters=70, random_state=0, n_init="auto").fit(features) # Number of cluster defined from elbow method.
    output= kmeans.labels_       # list of labels for elements occuring in each cluster 
    clusters = [np.squeeze(np.array(features)[[np.where(output==i)[0]]],axis=0) for i in range(len(np.unique(output)))] # Form clusters of deep features of image
    return kmeans.cluster_centers_, clusters

In [9]:
# Function that selects 100 data samples and removes the selected data from the dataset. (required to select balanced positive and negative samples)
def data_separation(dataset,label):
    add_data= []
    i=0
    while len(add_data)!=100:
        if dataset[i]["label"]==label:
            add_data.append(dataset[i]['image'])
            del dataset[i]
        i+=1
    return add_data

In [10]:
# Function that returns the mean of each subclusters. (required as mean is the representative of that subcluster)
def mean_features(positive, negative):
    # print(f"pure_pf: {positive}")
    # print(f"p_type: {type(positive)}")
    # print(f"len_p: {len(positive)}")
    mpos_features=np.array([np.mean(i,axis=0) for i in positive])  # Mean of all positive sub clusters 
    mneg_features=np.array([np.mean(i,axis=0) for i in negative])  # Mean of all negative sub clusters
    # print(mpos_features)
    return mpos_features, mneg_features

In [11]:
# Function that updates the subcluster by concatenating the test data sample to the most similar subcluster.
def update_subclusters(all_dist, query, fea_label, id_pred, label_pred, features, decision, n_neighbours, cluster):
    min_ind=np.argmin(all_dist)    # Get the index of the minimum distance to update the most similar (or nearest) subcluster.
    features[min_ind]=np.concatenate((features[min_ind],np.expand_dims(query["image"], axis=0)),axis=0)  # Concatenate on the most similar subcluster
    # fea_label[cluster].append(np.expand_dims(query["image"], axis=0))  # Have doubt here
    id_pred[cluster].append(query["id"])
    label_pred[cluster].append((query['id'],decision.count(1)/n_neighbours))  # decision.count(1)/n_neighbours --> Percentage that the model predict the data as positive (required to calculate AUC ROC value)
    return features, fea_label, id_pred, label_pred

In [12]:
# Function to check the model predicted label with the ground truth and corrects only if it is a mistake
def correct_mispredictions(query, fea_label,train_label, train_id, ind_data, decision,data_frame_1, count, pos_dist, neg_dist, pos_features, neg_features):
    
    if mode(decision) != query["label"]:  # Misclassification: if model's decision is different than the ground truth.
        # print("here")
        count +=1    
        data_frame_1["Image name"].append(query["filepath"].split("/")[-1])  # Recording to save it as csv file
        data_frame_1["Mistake ID"].append(query['id'])
        data_frame_1["Original label"].append(query['label'])
        data_frame_1["Predicted label"].append(mode(decision))
        data_frame_1["Mistake index"].append(ind_data)
        if query["label"]==1:
            pos_features_list= list(pos_features)  # converting numpy array into list to append 
            pos_features_list.append(np.expand_dims(query["image"], axis=0))   # Appending positive image to positive cluster (can use numpy function to do it without converting to list)
            pos_features = np.array(pos_features_list)
            # pos_features= np.concatenate((pos_features,np.expand_dims(query["image"], axis=0)),axis=0)
        else:
            neg_features_list= list(neg_features) # Same as above, but for negative class.
            neg_features_list.append(np.expand_dims(query["image"], axis=0))
            neg_features = np.array(neg_features_list)
            # neg_features = np.concatenate((neg_features,np.expand_dims(query["image"], axis=0)),axis=0)
        train_label[query['label']].append(query["label"])
        train_id[query['label']].append(query['id'])

    else:
        if query['label'] == 0:  # Correct classification by model: concatenating the feature to the closest subsample. 
            min_ind = np.argmin(neg_dist)  # Measuring the minimum distance with the representation (or mean) of subcluster, recording index of the nearest subcluster.
            neg_features[min_ind] = np.concatenate((neg_features[min_ind],np.expand_dims(query["image"], axis=0)),axis=0)
            # fea_label[query['label']].append(np.concatenate((fea_label[query['label']],np.expand_dims(query["image"], axis=0)),axis=0))
        else:
            min_ind = np.argmin(pos_dist)

            pos_features[min_ind] = np.concatenate((pos_features[min_ind],np.expand_dims(query["image"], axis=0)),axis=0)
            # fea_label[query['label']].append(np.concatenate((mpos_features,np.expand_dims(query["image"], axis=0)),axis=0))
        train_label[query['label']].append(query["label"])
        train_id[query['label']].append(query['id'])
    return count,data_frame_1,fea_label,train_label,train_id,pos_features,neg_features

In [13]:
# Function to measure distance of test data sample with each element in the training dataset (comparision occurs either with representative of subcluster or the element itself).
def distance2(query, fea_label, select_distance, id_pred, label_pred, n_neighbours, count, train_label, train_id, ind_data, data_frame_1, pos_features, neg_features, mentored_data): # Query is the raw dictionary (from pickle file) // fea_label is dictionary of {0: [], 1:[]} (distance) // select distance is int
  exp_query = np.expand_dims(query['image'], axis=0)
  pos_tup, neg_tup = [], []

  if select_distance==1: # Euclidean distance
    # print(f"Type: {type(fea_label[0])}")
    # print(f"Shape: {fea_label[0].shape}")
    neg_dist = np.linalg.norm(query['image']- fea_label[0], axis=1)  # Calculating the Euclidean distance using numpy (axis=1) to calculate all at ones   
    pos_dist = np.linalg.norm(query['image']- fea_label[1], axis=1)

  elif select_distance==2: # Manhattan distance
    neg_dist = np.squeeze(manhattan_distances(fea_label[0],exp_query))  # convert (1,n) to (,n), where n is the number of element in the list (or numpy array)
    pos_dist=np.squeeze(manhattan_distances(fea_label[1],exp_query))

  elif select_distance==3: # Cosine distance
    neg_dist = np.squeeze(cosine_distances(exp_query,fea_label[0]))  # convert (1,n) to (,n), where n is the number of element in the list (or numpy array)
    pos_dist=np.squeeze(cosine_distances(exp_query,fea_label[1]))
  
  for dist_single in pos_dist:  # Creating tuple (distance, label) to calculate AUC ROC value
    # print(dist_single)
    pos_tup.append((dist_single,1))

  for dist_single in neg_dist:    # Creating tuple (distance, label) to calculate AUC ROC value
    neg_tup.append((dist_single,0))

  pos_tup.extend(neg_tup)    # Extending the positive and negative tuple 
  tup_dist = sorted(pos_tup)[:n_neighbours]  # sorting the tuple according to distance to proceed with K Nearest Neighbor
  
  decision = [y for (x,y) in tup_dist]  # Decision contains list of labels

  if mentored_data:  # Check/compare the output of the model for mentored data. 
    count,data_frame_1,fea_label,train_label,train_id, pos_features,neg_features=correct_mispredictions(query, fea_label,train_label,train_id, ind_data, decision,data_frame_1, count, pos_dist, neg_dist, pos_features, neg_features)
    
  else: # For data sample other than the mentored, proceed with the following code. 
    if decision.count(0) > decision.count(1): 
      neg_features, fea_label, id_pred, label_pred = update_subclusters(neg_dist,query,fea_label,id_pred,label_pred,neg_features, decision, n_neighbours, cluster=0)
      
    else:
      pos_features, fea_label, id_pred, label_pred = update_subclusters(pos_dist,query,fea_label,id_pred,label_pred,pos_features, decision,n_neighbours, cluster=1)
  
  return id_pred, label_pred, data_frame_1, count, train_label, train_id, pos_features, neg_features

In [14]:
# Function to calculate the metrices.
def classification_metrics(label_gt,id_pred):
  TP,FP,FN,TN = 0,0,0,0

  for tp in id_pred[1]:   # TP --> When correctly classified covid #id_pred: dictionary
    if tp in label_gt[1]:
      TP +=1

  for tn in id_pred[0]:  # TN --> When correctly classified healthy (non-covid)
    if tn in label_gt[0]:
      TN +=1

  for fp in id_pred[1]: # FP --> When incorrectly classified healthy (Classified healthy as covid)
    if fp in label_gt[0]:
      FP +=1

  for fn in id_pred[0]: # FN --> When missed covid classification (Covid cases missed)
    if fn in label_gt[1]:
      FN +=1

  accuracy= (TP+TN)/(TP+TN+FP+FN)
  specificity = TN/(TN+FP)
  sensitivity = (TP)/(TP+FN)
  # f1_score = (2*precision*recall)/(precision + recall)
  
  print("TP: ", TP)
  print("FP: ", FP)
  print("FN: ", FN)
  print("TN: ", TN)

  return accuracy, specificity, sensitivity,TP,TN,FP,FN

def roc_auc_curve(label_gt,label_pred):
  gt_labels= sorted(label_gt[0]+ label_gt[1])  # Contains (id,labels) tuple of binary class 
  pred_labels = sorted(label_pred[0]+label_pred[1]) # Contains (id,labels) tuple of binary class --> sorted to match each element in gt_labels and pred_labels
  y_test = [y for (x,y) in gt_labels]   # Get only the labels
  y_scores = [y for (x,y) in pred_labels]
  fpr, tpr, threshold = roc_curve(y_test, y_scores)
  roc_auc = auc(fpr, tpr)
  return roc_auc

def cluster_metrics(pos_features, neg_features, train_label,id_pred):
  print("Calculating Dunn's index...")
  intra_dist1 = euclidean_distances(neg_features).max()
  intra_dist2 = euclidean_distances(pos_features).max()
  inter_dist = euclidean_distances(neg_features,pos_features).min()

  if intra_dist1>intra_dist2:
    max_intra_dist= intra_dist1  
  else:
    max_intra_dist = intra_dist2 

  Dunn_index = inter_dist/max_intra_dist

  print("Calculating Davies Bouldin index...")

  # Davies Bouldin and Silhouette score from sklearn library.
  # class_0 =np.concatenate((np.zeros(shape=(len(train_label[0])),dtype=int),np.zeros(shape=(len(id_pred[0])),dtype=int),np.zeros(shape=(20),dtype=int)))
  # class_1 = np.concatenate((np.ones(shape=(len(train_label[1])),dtype=int),np.ones(shape=(len(id_pred[1])),dtype=int),np.zeros(shape=(20),dtype=int)))
  class_0 =np.zeros(shape=(len(neg_features)),dtype=int)
  class_1 = np.ones(shape=(len(pos_features)),dtype=int)
  class_all = np.concatenate((class_0,class_1))
  feature_all = np.concatenate((neg_features,pos_features))

  davies_bouldin_index = davies_bouldin_score(feature_all,class_all)
  silhouette_index = silhouette_score(feature_all,class_all)

  print("davies: ", davies_bouldin_index)
  print("silhouette_sklearn: ", silhouette_index)
  
  return Dunn_index,davies_bouldin_index, silhouette_index

In [15]:
labeled_size = [400,800,1600,3200] # Labeled size consists of different mentored tiers (including the initial labeled data, e.g, labeled size of 400 consists of 200 labeled and 200 mentored data). 

# labeled_size = [3200]
def data_loader(dataset,n): # Method to return three sets of labeled dataset for experiment
  labeled_data, unlabeled_data = [], [] 

  l_data = dataset[:n]    # First case // labeled + mentored
  ul_data = dataset[n:]   # First case // unlabeled
  labeled_data.append(l_data)
  unlabeled_data.append(ul_data)

  l_data = dataset[3200:3200+n]    # Second case // labeled + mentored
  ul_data = dataset[:3200]+dataset[3200+n:]   # Second case // unlabeled
  labeled_data.append(l_data)
  unlabeled_data.append(ul_data)

  l_data = dataset[6400:6400+n]    # Third case // labeled + mentored
  ul_data = dataset[:6400]+dataset[6400+n:]     # Third case // unlabeled
  labeled_data.append(l_data)
  unlabeled_data.append(ul_data)
  return labeled_data, unlabeled_data

In [16]:
# Required to calculate the clustering indices
def flatten_features(features):
    all_features = []
    for i in features:
        for j in i:
            all_features.append(j)
    return all_features

In [17]:
n_neighbours=25

# Dataframe (dictionary) to save the result in csv format.
data_frame = {"Labeled data": [],
              "Dataset": [],
              "Accuracy": [],
              "Specificity": [],
              "Sensitivity": [],
              "AUC":[],
              "Dunn index": [],
              "Davies Bouldin": [],
              "Silhouette index":[],
              "TP":[],
              "TN":[],
              "FP":[],
              "FN":[],
              "pos_labeled_img":[],
              "neg_labeled_img":[],
              "corrected_count":[]
    
}
# fea_label1={0: [],
#             1:[]}


for size in labeled_size: # Select each size [400, 800,1600, 3200]
  labeled_data, unlabeled_data = data_loader(t_dataset, size)
#   print(f"labeled data length {len(labeled_data)}")
#   print(f"Unlabeled data length {len(unlabeled_data)}")
  select=0         # To select the dataset out of three cases ==> three cases: [d1, d2, d3] ==> eg: [3200,3200,3200]




  while(select < 3): # Save the mistake information as csv file
    data_frame_1 = {  "Image name": [],
                  "Mistake index": [],
                  "Mistake ID": [],
                  "Original label": [],
                  "Predicted label": []
                  
    }
    pos_img, neg_img=0, 0

    fpos, fneg= [], []   

    label_gt = {0: [],    # key: 0 & 1 (class), value: tuple (data['id'],data['label']), required to calulate TP, FP, FN, TN
        1 :[]}    
                            
    train_label = {0: [],    # # key: 0 & 1 (class), value: ground truth labels
        1 :[]}    

    label_pred = {0: [],  # key: 0 & 1 (class), value: tuple(query['id'],decision.count(1)/n_neighbours) --> decision.count(1)/n_neighbours: Percentage of predicted positive class, required to calculate AUC/roc value
        1 :[]}              

    id_gt = {0: [], #  key: 0 & 1 (class), value: ground truth id  (Collect the ground truth (id) of all the predicting images)
            1: [] }         
    
    id_pred = {0: [],  #  key: 0 & 1 (class), value: predicted ids, required to calulate TP, FP, FN, TN
               1: []
               }
    fea_label = {0: [],  #  key: 0 & 1 (class), value: deep feature of image
            1: []}

    train_id ={0: [],  #  key: 0 & 1 (class), value: id of images   --> Not required for , but scared to delete. 
            1:[]}
        
    # print(type(labeled_data[0][0]))
    # for data in labeled_data[select]:
    #     if data["label"] == 1:
    #         fpos.append(data['image'])
    #         train_id[1].append(data['id'])
    #         train_label[1].append((data['id'],data['label']))
    #         pos_img +=1

    #     else:
    #         fneg.append(data['image'])
    #         train_id[0].append(data['id'])
    #         train_label[0].append((data['id'],data['label']))
    #         neg_img +=1

    # print(f"Blen: {len(labeled_data[select])}")
    
    # Select 200 labeled data
    fpositive = data_separation(labeled_data[select],1)    # Get the balanced positive features
    fnegative = data_separation(labeled_data[select],0)  # Get the balanced negative features


    mneg_features,neg_features= sub_clusters(fnegative)  # Get the subclusters (Using K-means algorithm)
    mpos_features,pos_features= sub_clusters(fpositive)  # Get the subclusters (Using K-means algorithm)

        

    count, ind_data=0, 200    # Count to record the number of mistakes // ind_data: required to keep track of the indexes the model makes mistakes.
    
    # This loop is for the mentored data --> Notice mentored_data = True in argument of the function call distance2().
    for data in labeled_data[select]:
        fea_label={0: mneg_features,
            1: mpos_features}
        
        id_pred, label_pred, data_frame_1, count, train_label, train_id, pos_features, neg_features= distance2(data,fea_label,3,id_pred,label_pred,n_neighbours, count, train_label, train_id, ind_data, data_frame_1, pos_features, neg_features, mentored_data=True)
        mpos_features, mneg_features = mean_features(pos_features, neg_features)    # Get the mean of the features
        ind_data +=1

    data_f_1 = pd.DataFrame.from_dict(data_frame_1)
    data_f_1.to_csv(f"../subclusters_csv/ct-scans/mistake/densenet169_cosine_mistake_{size}_{select}.csv",index=False) # Save the misclassiciation records

   # This loop is for the test data --> Notice mentored_data = False in argument of the function call distance2().
    for data in tqdm(unlabeled_data[select]):
      if data["label"]==1:
        id_gt[1].append(data['id'])
        label_gt[1].append((data['id'],data['label']))   # Required to calulate TP, FP, FN, TN
      
      else:
        id_gt[0].append(data['id'])
        label_gt[0].append((data['id'],data['label']))
      
      fea_label={0: mneg_features,
            1: mpos_features}

      id_pred, label_pred, _, _, _, _, pos_features, neg_features = distance2(data,fea_label,3,id_pred,label_pred,n_neighbours, count, train_label, train_id, ind_data, data_frame_1, pos_features, neg_features,mentored_data=False) # ind_data is the index of misclassification
      mpos_features, mneg_features = mean_features(pos_features, neg_features)    # Get the mean of the features

    accuracy, specificity, sensitivity,TP,TN,FP,FN= classification_metrics(id_gt,id_pred)
    flattened_pos_features = flatten_features(pos_features) # Flattened as required to calculate clustering indices.
    flattened_neg_features = flatten_features(neg_features)
    dunn_index, davies_bouldin_index, silhouette_index = cluster_metrics(flattened_pos_features, flattened_neg_features, train_label,id_pred)
    cl_auc = roc_auc_curve(label_gt,label_pred)
    
    data_frame["Labeled data"].append(size)
    data_frame["Dataset"].append(f"d_{select}")
    data_frame["Accuracy"].append(accuracy)
    data_frame["Specificity"].append(specificity)
    data_frame["Sensitivity"].append(sensitivity)
    data_frame["AUC"].append(cl_auc)
    data_frame["Dunn index"].append(dunn_index)
    data_frame["Davies Bouldin"].append(davies_bouldin_index)
    data_frame["Silhouette index"].append(silhouette_index)
    data_frame["TP"].append(TP)
    data_frame["TN"].append(TN)
    data_frame["FP"].append(FP)
    data_frame["FN"].append(FN)
    data_frame["pos_labeled_img"].append(pos_img)
    data_frame["neg_labeled_img"].append(neg_img)
    data_frame["corrected_count"].append(count)

    print(f"Labeled image: {size} \t Dataset: d_{select} \t Accuracy: {accuracy} \t Specificity: {specificity} \t Sensitivity: {sensitivity} \t Dunn index: {dunn_index}  \t Davies Bouldin: {davies_bouldin_index} \t Silhouette index: {silhouette_index} \t AUC: {cl_auc} \t Corrected count: {count}")
    select +=1 

    data_f=pd.DataFrame.from_dict(data_frame)
    data_f.to_csv(f"../subclusters_csv/ct-scans/densenet169_cosine_dist.csv",index=False)
  

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (71,) + inhomogeneous part.